In [21]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

## Group Assignment
### Team Number: XX
### Team Member Names:
### Team Strategy Chosen: __________(RISKY OR SAFE)

In [ ]:
## Assign csv data to a variable
col_names=['AAPL']
tick_data = pd.read_csv("Tickers.csv", usecols=col_names)
print(tick_data['AAPL'])

In [48]:
## Assign csv data to a variable
col_names=['Ticker']
tick_data = pd.read_csv("Tickers.csv", names=col_names, header=None)

0     ABBV
1      ABT
2      ACN
3      AGN
4      AIG
      ... 
56     TXN
57     UNH
58     UNP
59     UPS
60     USB
Name: AAPL, Length: 61, dtype: object


In [72]:
# Remove delisted tickers 
remove_delisted = []

for x in range(len(tick_data.index)):
    tick = yf.Ticker(tick_data['AAPL'].iloc[x])
    hist = tick.history()
    if hist.empty == False :
        remove_delisted.append(tick_data['AAPL'].iloc[x])
        
remove_delisted = list(dict.fromkeys(delisted_tickers))
tickers = pd.DataFrame(delisted_tickers)
tickers.head()

- AGN: No data found, symbol may be delisted
- CELG: No data found, symbol may be delisted
- PCLN: No data found for this date range, symbol may be delisted
- RTN: No data found, symbol may be delisted
- TWX: No data found for this date range, symbol may be delisted


,0
0,ABBV
1,ABT
2,ACN
3,AIG
4,AMZN


In [ ]:
temp_list = []
tick_list = []
for x in range(0, len(tickers.index)):
    tick = yf.Ticker(tickers.iloc[x][0])
    info = tick.info
    data = pd.DataFrame(info.keys())
    data.columns = ['Value']
    if data[data.Value == 'country'].empty == False :
        temp_list.append(tick_data['AAPL'].iloc[x])
print(temp_list)

start_date = '2021-07-02'
end_date = '2021-10-22'
for x in range(len(tick_list.index)):
    tick = yf.Ticker(tick_list[x])
    tick_hist = tick.history(start=start_date, end=end_date)
    avg_volume = tick_hist['Volume'].sum(axis = 0, skipna = True)/ len(tick_hist.index)
    if avg_volume >= 10000:
        tick_list.append(tick)
        
print(tick_list)

In [ ]:
for x in range (len(tick_list)):
    tick = yf.Ticker(tick_data['Ticker'].iloc[x])
    

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.